# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit 0ae2204440025d65939c8397c8c2780b43569879
Author: Najwa Ez Zine <nezzine@frennes>
Date:   Wed Jun 20 15:40:17 2018 +0200

    Automatic results update : Time_Bcast---rennes Fix  : Comments


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo
pip3 install --user requests
pip install rpy2

  Using cached https://files.pythonhosted.org/packages/65/47/7e02164a2a3db50ed6d8a6ab1d6d60b69c4c3fdf57a284257925dfc12bda/requests-2.19.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bd/c9/6fdd990019071a4a32a5e7cb78a1d92c53851ef4f56f62a3486e6a7d8ffb/urllib3-1.23-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a97fab05a372e4a2c6a6b876165/idna-2.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/88/f7/788e5e1587bc1ab48b5b0467f9c3a08ef5aadbd993ed835559a10e452121/rpy2-2.9.3.tar.gz
    Complete output from command python setup.py egg_info:
    rpy2 is no l

Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-r7g4uJ/rpy2/


In [4]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output
import random
from threading import Thread
import math
import collections
from collections import deque

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 3
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou')\"" 

# Nancy
site = "rennes"

In [6]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1017043


On les range

In [7]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('parasilo-26.rennes.grid5000.fr'),
 Host('parasilo-27.rennes.grid5000.fr'),
 Host('parasilo-28.rennes.grid5000.fr')]

##### Déploiement de l'environnement

In [8]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [9]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

rennes: Deployment #D-f5c6e430-7e9a-4749-afb0-aeca73e1f754 started
rennes: Grab the key file /home/nezzine/.ssh/authorized_keys
rennes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
rennes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
rennes: Launching a deployment on parasilo-[26-28].rennes.grid5000.fr
rennes: Performing a Deploy[SetDeploymentEnvUntrusted] step
rennes:   switch_pxe
rennes:   reboot
rennes:    * Performing a soft reboot on parasilo-[26-28].rennes.grid5000.fr
rennes:   wait_reboot
rennes:   send_key_in_deploy_env
rennes:   create_partition_table
rennes:   format_deploy_part
rennes:   mount_deploy_part
rennes:   format_tmp_part
rennes:   format_swap_part
rennes: End of step Deploy[SetDeploymentEnvUntrusted] after 196s
rennes: Performing a Deploy[BroadcastEnvKascade] step
rennes:   send_environment
rennes:    * Broadcast time: 71s
rennes:   manage_admin_post_install
rennes:   manage_user

rennes: Warning: Permanently added 'rennes.grid5000.fr,172.16.111.106' (ECDSA) to the list of known hosts.
rennes: Connection to rennes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [10]:
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

True

#### Installation environnement sur noeud

In [11]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [12]:
Remote_install.run().ok

True

#### Version Simgrid

In [13]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [14]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [15]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

In [16]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [17]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        line = l[i].split()
        S[int(line[1])] = int(float(line[5]))
    return S

In [18]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return [D,min(S)]

In [ ]:
# Example
delayer(0,10,'2018-06-04_12:38')

##### Shortcut : séquentiel

In [19]:
def setup() :
    Src_dir = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
    Src_get = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
    Rslt_dir = \
        execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

# On récup les src
    Src_dir.run().ok
    Src_get.run().ok
    Rslt_dir.run().ok


In [ ]:
# Test : trajectoire jusqu'à la ième machine raccrochée à la trajectoire initiale
def coherence(tab,i):
     return all(x == True for x in tab[0:i])

In [ ]:

def parl_exec () :
    setup()
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Parallel simulation

In [20]:
# Thread pour la simulation // opt.
class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""
    
    # Paramètrage du thread
    def __init__(self,i,MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        self.i = i
    
    # Set la valeur du délai
    def set_delay(self,delay):
        self.DELAY = delay
    
    def get_delay(self):
        return self.DELAY
    def get_MIN(self):
        return self.MIN
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        """print(get_host_shortname(nodes[self.i])+" => "+str(self.i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'\n"
            )
        """
        # make run sur toutes la machine dont le thread s'occupe avec ses paramètres
        execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[self.i],connection_params={'user':'root'}).run().ok


In [21]:
# Formate le tableau en string pour pouvoir etre passé en ligne de commande
def format_cmd(data):
    return (' '.join(str(e) for e in data))

In [22]:
# Formate le string en tableau d'int pour être manipulé 
def unformat_cmd(data):
    return list(map(int,data.split()))

In [23]:
# SOURCE : https://stackoverflow.com/questions/3229419/how-to-pretty-print-nested-dictionaries
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, type({})):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [24]:
def color_code(color):
    if (color == "b_blue"):
        code = 94 
    elif (color == "green"):
        code = 32
    elif (color == "red" ):
        code = 31
    elif (color == "b_red" ):
        code = 91
    elif (color == "yellow"):
        code = 33
    elif (color == "b_yellow"):
        code = 93
    elif (color == "magenta"):
        code = 35
    elif (color == "cyan"):
        code = 36
    elif (color =="b_black"):
        code = 90
    elif (color =="black"):
        code = 30
    elif (color == "blue"):
        code = 34
    elif (color == "white"):
        code = 97
    else :
        code = 0
    return code

def printc (string,color,*args) :
    bold = 1
    try :
        background = args[0]
    except IndexError :
        background = 0
    try :
        additional = args[1]
    except :
        additional =""
    
    code = color_code(color)
    background = color_code(background) + 10
    
    print("\x1b["+str(bold)+";"+str(code)+";"+str(background)+"m"+ string+"\x1b[0m"+ additional, flush=True)

In [25]:
printc("ROUND 1 ","red",None, "Hello")

ROUND 1 Hello


In [26]:
def generate_Dprime(NPROCS,border_inf,border_sup):
    new_Dprime = []
    if (border_inf == 0):
        for p in range(NPROCS):
            new_Dprime.append(random.choice(range(border_inf,border_sup[p]+1)))
    else : 
        for p in range(NPROCS):
            new_Dprime.append(random.choice(range(border_inf[p]+1,border_sup[p]+1)))
    return format_cmd(new_Dprime)

In [68]:
# SIMULATION
def simulation(N, NPROCS, fragment_size, mode, catch_me):
    """Contrôle de la validité des paramètres"""
    # Taille du fragment ne peut être supérieure à la zone d'itération
    if (fragment_size > N): 
        raise ValueError("Fragment size cannot be superior to N")
        
    """Mise en place de l'environnement et des variables"""
    # Environnement
    setup() 
    
    # Dates: pour la gestion de fichiers (IMPORTANT pour lecture/écriture)
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    date = date_precise.split('_')[0]
    
    # Total execution time
    total_execution_time = 0
    
    # Tableau de threads
    machines = init_tab(nodecount, None)
      
    # position trajectoire cohérente courante
    t_curr = 0
    
    # D de la trajectorie cohérente courante
    D_curr = format_cmd(init_tab(NPROCS, 0))
    
    # Taille du dernier slot
    last_fragment_size = N % fragment_size
    
    # Nombre de fragments
    fragment_number = int((math.ceil(N / fragment_size)))
    
    # Nombre de rounds
    rounds_taken = 0
    
    # zone d'itérations à explorer
    To_explore = collections.deque()
    
    # Initialisation des zones à explorer D = '0...0' pour chaque zone T -> (Min d'itération , D initial)
    for i in range(fragment_number):
        To_explore.append((i * fragment_size, format_cmd(init_tab(NPROCS, 0))))

    # Round actuel
    curr_round = 0
    
    # Liste des trajectoires pour un temps t et un D donnée
    sim = {}
    
    success = False 
    
    while(True):
        curr_round = curr_round + 1
        
        # Trajectoire cohérente complète trouvé : FIN DE SIMULATION
        if (t_curr >= N) :
            printc ("Total time : " + str(total_execution_time) , "black" , "yellow")
            printc ("Success : " + str(success) , "black" , "green")
            break;        
        if (len(To_explore) == 0):
            raise 
        # Il n'existe pas encore de trajectoire cohérente -> Nouveau round
        printc("--------------------------------------------  ROUND :  " + str(curr_round) + "   --------------------------------------------\n","red","white")
        
        # Zones restantes à explorer
        number_paths_To_explore = len(To_explore)
        
        # Nombre de tâches parallèles lançables
        number_launchable_tasks = min(nodecount, number_paths_To_explore)
        
        # Affichage des paramètres
        printc ("  || LAUNCHING || \n","green")
        printc ( "TO EXPLORE : " , "blue" , None, str(To_explore)) 
        
        """ Paramètrage des machines """
        
        # Soit j'ai plus de zone à explorer que de machines disponibles soit l'inverse
        for q in range(number_launchable_tasks):
            
            # On récupère la première zone à explorer
            parameters = To_explore.popleft()
            
            # MIN est le champ de gauche
            MIN = parameters[0]
            
            # MAX de la zone à explorer vaut le min + la taille du slot
            MAX = min (MIN + fragment_size, N)
            
            # D est le champ de droite
            delay = parameters[1]
            
            # Paramètrage des machines
            machines[q] = ParSimer(q, MIN, MAX, N, delay)
            #print("Machine " + str(q) + " -> MIN :" + str(MIN) + " MAX : " + str(MAX) + " D : "+ delay)   
                
        """ Simulation """  
        # Lancement de la simulation
        for q in range(number_launchable_tasks):
            machines[q].start()
            
        # Fin de simulation
        for q in range(number_launchable_tasks) :
            machines[q].join()
            # Récupération des résultats
            execo.action.Get(hosts=nodes[q],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[q]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok    
            
        """ Analyse des résultats"""    
        # Traitement des données
        for q in range(number_launchable_tasks):
            # On récupère les paramètres des machines
            t = machines[q].get_MIN()
            D = machines[q].get_delay()
            
            # Calcul du délai
            # On récupère D et S
            raw_data = delayer(q, NPROCS, date_precise)
            
            # Filtre D : Calcul du délai
            Dprime = format_cmd(raw_data[0])
            
            #Filte S : Calcul du temps d'éxécution total 
            T = raw_data[1]
            
            # Ajout dans ma table (Dprime, round, machine qui l'a calculé, temps)
            sim.setdefault(t,{}).update({D:(Dprime,curr_round,q,T)})
            MAX = min (t + fragment_size, N)
        
        # Affichage des paramètres
        printc ("T_curr : " , "yellow" , None , str(t_curr))
        printc ("D_curr : " , "yellow" , None , str(D_curr))
        printc ("Dprime : ", "b_yellow" , None , str(Dprime))
        printc ("total execution time : ", "red" , None, str(total_execution_time) + "\n") 
        
        merged_paths = 0
        # Calcul de la trajectoire cohérente
        while (t_curr < N and D_curr in sim[t_curr]):
            merged_paths += 1
            # Update le temps d'exécution
            total_execution_time += sim[t_curr][D_curr][3]
            
            # Update du temps d'exécution dans le dernier fragment
            if (t_curr + fragment_size >= N) : total_execution_time += max(unformat_cmd(sim[t_curr][D_curr][0]))
            
            # Update des t_curr et D_curr
            D_curr = sim[t_curr][D_curr][0]
            t_curr = min(t_curr + fragment_size, N)
        if (merged_paths > 1) : 
            printc("****** SUCCESS : ", "red", "black", str(merged_paths))
            success = True 
        
        # Affichage des résultats
        printc ("\n  || RESULTS || \n","red")
        printc ( "TO EXPLORE : " , "blue" , None, str(To_explore)) 
        pretty (sim) 
        printc ("T_curr : " , "yellow" , None , str(t_curr))
        printc ("D_curr : " , "yellow" , None , str(D_curr))
        printc ("Dprime : ", "b_yellow" , None , str(Dprime))
        printc ("total execution time : ", "b_red" , None, str(total_execution_time) + "\n") 
        
        
        """ Nouveau paramètrages """
        printc("  || ANALYSIS || \n","magenta")
        
        for q in range(number_launchable_tasks):
            t = machines[q].get_MIN()
            D = machines[q].get_delay()
            
            MAX = min(t + fragment_size, N)
            Dprime = sim[t][D][0]
            catch_me_ind = 0
            # Zone valide
            if(MAX < N):
                # Zone pas encore relié à la trajectoire initiale
                if (MAX > t_curr):
                    caught = False
                # Ajout dans les zones à explorer du nouveau point de départ à exploiter en fonction du mode
                    # Test catch me sur 2^catch_me_ind round
                    if (catch_me and 1 << catch_me_ind == curr_round):
                        catch_me_ind += 1
                        try :
                            caught = Dprime in sim[MAX]
                        except KeyError:
                            pass 
                       
                    if (not caught):
                        borders = unformat_cmd(Dprime)
                        double_Dprime = [(2*(x+1)) for x in borders]
                      
                        # Bornes pour le random 
                        border_inf = 0
                        border_sup = double_Dprime
                        if (mode == "RAND_SUP") : 
                            border_sup = borders
                            Dprime =  generate_Dprime(NPROCS, border_inf, border_sup)
                        elif (mode == "RAND_INF") : 
                            border_inf = borders
                            Dprime = generate_Dprime(NPROCS, border_inf, border_sup)
                        elif (mode == "RAND") : 
                            Dprime =  generate_Dprime(NPROCS, border_inf, border_sup)
                        print("["+str(MAX) +"] Generated " + str(mode) + " Dprime : " + str(Dprime))
                To_explore.append((MAX, Dprime))
        printc ( "TO EXPLORE : " , "blue" , None, str(To_explore) + "\n")
        

In [71]:
# Nbr pcs
NPROCS = 3
    
# Nbr itérations
N = 100

# taille paquet d'itérations
fragment_size = 5
    

# mode de génération de l'état initial à chaque round
mode_opt = ["FIX","RAND","RAND_INF","RAND_SUP"]
mode = mode_opt[1]

catch_me = False

# Lancement de la simulation
simulation(N,NPROCS,fragment_size,mode,catch_me)

--------------------------------------------  ROUND :  1   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(0, '0 0 0'), (5, '0 0 0'), (10, '0 0 0'), (15, '0 0 0'), (20, '0 0 0'), (25, '0 0 0'), (30, '0 0 0'), (35, '0 0 0'), (40, '0 0 0'), (45, '0 0 0'), (50, '0 0 0'), (55, '0 0 0'), (60, '0 0 0'), (65, '0 0 0'), (70, '0 0 0'), (75, '0 0 0'), (80, '0 0 0'), (85, '0 0 0'), (90, '0 0 0'), (95, '0 0 0')])
T_curr : 0
D_curr : 0 0 0
Dprime : 0 0 0
total execution time : 0


  || RESULTS || 

TO EXPLORE : deque([(15, '0 0 0'), (20, '0 0 0'), (25, '0 0 0'), (30, '0 0 0'), (35, '0 0 0'), (40, '0 0 0'), (45, '0 0 0'), (50, '0 0 0'), (55, '0 0 0'), (60, '0 0 0'), (65, '0 0 0'), (70, '0 0 0'), (75, '0 0 0'), (80, '0 0 0'), (85, '0 0 0'), (90, '0 0 0'), (95, '0 0 0')])
0
	0 0 0
		('2 3 0', 1, 0, 0)
10
	0 0 0
		('0 0 0', 1, 2, 2)
5
	0 0 0
		('0 0 0', 1, 1, 2)
T_curr : 5
D_curr : 2 3 0
Dprime : 0 0 0
total execution time : 0

  || ANALYSIS || 

[10] Generated RA

D_curr : 2 3 0
Dprime : 0 2 1
total execution time : 0

  || ANALYSIS || 

[65] Generated RAND Dprime : 2 2 0
[70] Generated RAND Dprime : 1 5 3
[75] Generated RAND Dprime : 0 3 3
TO EXPLORE : deque([(75, '0 0 0'), (80, '0 0 0'), (85, '0 0 0'), (90, '0 0 0'), (95, '0 0 0'), (5, '2 3 0'), (10, '0 1 0'), (15, '0 0 0'), (20, '2 3 5'), (25, '0 5 6'), (30, '1 0 1'), (35, '2 2 2'), (40, '2 0 0'), (45, '5 2 1'), (50, '0 1 1'), (55, '2 2 2'), (60, '0 2 1'), (65, '2 2 0'), (70, '1 5 3'), (75, '0 3 3')])

--------------------------------------------  ROUND :  6   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(75, '0 0 0'), (80, '0 0 0'), (85, '0 0 0'), (90, '0 0 0'), (95, '0 0 0'), (5, '2 3 0'), (10, '0 1 0'), (15, '0 0 0'), (20, '2 3 5'), (25, '0 5 6'), (30, '1 0 1'), (35, '2 2 2'), (40, '2 0 0'), (45, '5 2 1'), (50, '0 1 1'), (55, '2 2 2'), (60, '0 2 1'), (65, '2 2 0'), (70, '1 5 3'), (75, '0 3 3')])
T_curr : 5
D_curr : 2 3 0
Dprime : 0 0 0
total executi

D_curr : 3 2 0
Dprime : 2 0 1
total execution time : 2

  || ANALYSIS || 

[30] Generated RAND Dprime : 2 1 15
[35] Generated RAND Dprime : 2 3 4
[40] Generated RAND Dprime : 1 1 4
TO EXPLORE : deque([(40, '2 0 0'), (45, '5 2 1'), (50, '0 1 1'), (55, '2 2 2'), (60, '0 2 1'), (65, '2 2 0'), (70, '1 5 3'), (75, '0 3 3'), (80, '0 1 0'), (85, '1 0 1'), (90, '2 2 2'), (95, '1 1 2'), (10, '3 2 0'), (15, '0 0 4'), (20, '0 0 1'), (25, '1 2 0'), (30, '2 1 15'), (35, '2 3 4'), (40, '1 1 4')])

--------------------------------------------  ROUND :  10   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(40, '2 0 0'), (45, '5 2 1'), (50, '0 1 1'), (55, '2 2 2'), (60, '0 2 1'), (65, '2 2 0'), (70, '1 5 3'), (75, '0 3 3'), (80, '0 1 0'), (85, '1 0 1'), (90, '2 2 2'), (95, '1 1 2'), (10, '3 2 0'), (15, '0 0 4'), (20, '0 0 1'), (25, '1 2 0'), (30, '2 1 15'), (35, '2 3 4'), (40, '1 1 4')])
T_curr : 10
D_curr : 3 2 0
Dprime : 0 0 1
total execution time : 2


  || RESU

T_curr : 10
D_curr : 3 2 0
Dprime : 0 0 1
total execution time : 2


  || RESULTS || 

TO EXPLORE : deque([(10, '3 2 0'), (15, '0 0 4'), (20, '0 0 1'), (25, '1 2 0'), (30, '2 1 15'), (35, '2 3 4'), (40, '1 1 4'), (45, '5 0 0'), (50, '7 8 1'), (55, '2 1 2'), (60, '2 1 2'), (65, '2 4 5'), (70, '3 3 0'), (75, '1 1 0'), (80, '0 10 2'), (85, '2 8 2')])
0
	0 0 0
		('2 3 0', 1, 0, 0)
65
	0 0 0
		('0 2 1', 5, 1, 1)
	2 2 0
		('1 2 0', 11, 2, 2)
35
	0 0 0
		('0 0 0', 3, 1, 2)
	2 2 2
		('2 0 1', 9, 2, 3)
5
	0 0 0
		('0 0 0', 1, 1, 2)
	2 3 0
		('3 2 0', 7, 2, 2)
70
	0 0 0
		('0 2 1', 5, 2, 1)
	1 5 3
		('1 5 0', 12, 0, 4)
40
	0 0 0
		('2 1 0', 3, 2, 1)
	2 0 0
		('2 0 2', 10, 0, 2)
10
	0 1 0
		('1 0 1', 8, 0, 2)
	0 0 0
		('0 0 0', 1, 2, 2)
75
	0 0 0
		('0 0 0', 6, 0, 3)
	0 3 3
		('0 5 2', 12, 1, 3)
45
	0 0 0
		('0 1 1', 4, 0, 1)
	5 2 1
		('4 3 0', 10, 1, 3)
15
	0 0 0
		('0 0 1', 8, 1, 2)
80
	0 1 0
		('0 3 1', 12, 2, 1)
	0 0 0
		('0 0 0', 6, 1, 3)
50
	0 0 0
		('1 1 0', 4, 1, 1)
	0 1 1
		('0 0 1', 10,

0
	0 0 0
		('2 3 0', 1, 0, 0)
65
	0 0 0
		('0 2 1', 5, 1, 1)
	2 2 0
		('1 2 0', 11, 2, 2)
35
	0 0 0
		('0 0 0', 3, 1, 2)
	2 3 4
		('0 1 2', 15, 2, 5)
	2 2 2
		('2 0 1', 9, 2, 3)
5
	0 0 0
		('0 0 0', 1, 1, 2)
	2 3 0
		('3 2 0', 7, 2, 2)
70
	0 0 0
		('0 2 1', 5, 2, 1)
	1 5 3
		('1 5 0', 12, 0, 4)
40
	1 1 4
		('0 1 3', 16, 0, 3)
	0 0 0
		('2 1 0', 3, 2, 1)
	2 0 0
		('2 0 2', 10, 0, 2)
10
	0 1 0
		('1 0 1', 8, 0, 2)
	0 0 0
		('0 0 0', 1, 2, 2)
	3 2 0
		('8 5 0', 14, 0, 0)
75
	0 0 0
		('0 0 0', 6, 0, 3)
	0 3 3
		('0 5 2', 12, 1, 3)
45
	5 0 0
		('4 1 0', 16, 1, 2)
	0 0 0
		('0 1 1', 4, 0, 1)
	5 2 1
		('4 3 0', 10, 1, 3)
15
	0 0 0
		('0 0 1', 8, 1, 2)
	0 0 4
		('2 0 7', 14, 1, 1)
80
	0 1 0
		('0 3 1', 12, 2, 1)
	0 0 0
		('0 0 0', 6, 1, 3)
50
	7 8 1
		('4 4 0', 16, 2, 5)
	0 0 0
		('1 1 0', 4, 1, 1)
	0 1 1
		('0 0 1', 10, 2, 3)
20
	0 0 0
		('0 2 2', 2, 1, 1)
	0 0 1
		('0 2 4', 14, 2, 1)
	2 3 5
		('0 0 0', 8, 2, 6)
85
	1 0 1
		('0 0 1', 13, 0, 3)
	0 0 0
		('0 0 0', 6, 2, 3)
55
	0 0 0
		('1 1 0',

...

D_curr : 14 15 0
Dprime : 0 8 42
total execution time : 25

  || ANALYSIS || 

[90] Generated RAND Dprime : 0 5 8
[95] Generated RAND Dprime : 2 3 55
TO EXPLORE : deque([(70, '14 15 0'), (75, '34 0 1'), (80, '0 15 1'), (85, '3 1 57'), (90, '0 5 8'), (95, '2 3 55')])

--------------------------------------------  ROUND :  64   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(70, '14 15 0'), (75, '34 0 1'), (80, '0 15 1'), (85, '3 1 57'), (90, '0 5 8'), (95, '2 3 55')])
T_curr : 70
D_curr : 14 15 0
Dprime : 0 11 1
total execution time : 25


  || RESULTS || 

TO EXPLORE : deque([(85, '3 1 57'), (90, '0 5 8'), (95, '2 3 55')])
0
	0 0 0
		('2 3 0', 1, 0, 0)
65
	10 2 1
		('9 2 0', 59, 1, 3)
	0 0 0
		('0 2 1', 5, 1, 1)
	2 2 0
		('1 2 0', 11, 2, 2)
	2 4 5
		('0 5 3', 17, 2, 4)
	1 14 3
		('0 15 3', 33, 2, 2)
	2 1 1
		('2 0 0', 28, 2, 4)
	0 29 3
		('0 30 5', 56, 2, 1)
	2 0 5
		('0 0 3', 42, 2, 3)
	10 2 62
		('7 0 61', 53, 2, 5)
	0 4 0
		('0 6 2', 50, 1, 2)


D_curr : 13 12 0
Dprime : 0 2 57
total execution time : 29

  || ANALYSIS || 

[90] Generated RAND Dprime : 2 1 18
[95] Generated RAND Dprime : 2 9 12
TO EXPLORE : deque([(75, '13 12 0'), (80, '62 2 2'), (85, '0 7 2'), (90, '2 1 18'), (95, '2 9 12')])

--------------------------------------------  ROUND :  66   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(75, '13 12 0'), (80, '62 2 2'), (85, '0 7 2'), (90, '2 1 18'), (95, '2 9 12')])
T_curr : 75
D_curr : 13 12 0
Dprime : 0 6 5
total execution time : 29


  || RESULTS || 

TO EXPLORE : deque([(90, '2 1 18'), (95, '2 9 12')])
0
	0 0 0
		('2 3 0', 1, 0, 0)
65
	10 2 1
		('9 2 0', 59, 1, 3)
	0 0 0
		('0 2 1', 5, 1, 1)
	2 2 0
		('1 2 0', 11, 2, 2)
	2 4 5
		('0 5 3', 17, 2, 4)
	1 14 3
		('0 15 3', 33, 2, 2)
	2 1 1
		('2 0 0', 28, 2, 4)
	0 29 3
		('0 30 5', 56, 2, 1)
	2 0 5
		('0 0 3', 42, 2, 3)
	10 2 62
		('7 0 61', 53, 2, 5)
	0 4 0
		('0 6 2', 50, 1, 2)
	0 6 0
		('0 7 1', 23, 1, 2)
	1 3 1
		('2 2 0',

D_curr : 12 13 0
Dprime : 12 13 0
total execution time : 34

  || ANALYSIS || 

[95] Generated RAND Dprime : 2 0 15
TO EXPLORE : deque([(85, '1 0 2'), (90, '1 14 11'), (95, '2 0 15'), (85, '12 13 0')])

--------------------------------------------  ROUND :  68   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(85, '1 0 2'), (90, '1 14 11'), (95, '2 0 15'), (85, '12 13 0')])
T_curr : 85
D_curr : 12 13 0
Dprime : 0 0 12
total execution time : 34


  || RESULTS || 

TO EXPLORE : deque([(85, '12 13 0')])
0
	0 0 0
		('2 3 0', 1, 0, 0)
65
	10 2 1
		('9 2 0', 59, 1, 3)
	0 0 0
		('0 2 1', 5, 1, 1)
	2 2 0
		('1 2 0', 11, 2, 2)
	2 4 5
		('0 5 3', 17, 2, 4)
	1 14 3
		('0 15 3', 33, 2, 2)
	2 1 1
		('2 0 0', 28, 2, 4)
	0 29 3
		('0 30 5', 56, 2, 1)
	2 0 5
		('0 0 3', 42, 2, 3)
	10 2 62
		('7 0 61', 53, 2, 5)
	0 4 0
		('0 6 2', 50, 1, 2)
	0 6 0
		('0 7 1', 23, 1, 2)
	1 3 1
		('2 2 0', 38, 1, 4)
	14 14 0
		('14 15 0', 61, 2, 2)
	1 0 6
		('0 2 7', 46, 2, 2)
35
	0 

D_curr : 14 12 0
Dprime : 0 8 12
total execution time : 36

  || ANALYSIS || 

[95] Generated RAND Dprime : 3 0 1
TO EXPLORE : deque([(90, '14 12 0'), (95, '3 0 1')])

--------------------------------------------  ROUND :  70   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE : deque([(90, '14 12 0'), (95, '3 0 1')])
T_curr : 90
D_curr : 14 12 0
Dprime : 5 0 5
total execution time : 36


  || RESULTS || 

TO EXPLORE : deque([])
0
	0 0 0
		('2 3 0', 1, 0, 0)
65
	10 2 1
		('9 2 0', 59, 1, 3)
	0 0 0
		('0 2 1', 5, 1, 1)
	2 2 0
		('1 2 0', 11, 2, 2)
	2 4 5
		('0 5 3', 17, 2, 4)
	1 14 3
		('0 15 3', 33, 2, 2)
	2 1 1
		('2 0 0', 28, 2, 4)
	0 29 3
		('0 30 5', 56, 2, 1)
	2 0 5
		('0 0 3', 42, 2, 3)
	10 2 62
		('7 0 61', 53, 2, 5)
	0 4 0
		('0 6 2', 50, 1, 2)
	0 6 0
		('0 7 1', 23, 1, 2)
	1 3 1
		('2 2 0', 38, 1, 4)
	14 14 0
		('14 15 0', 61, 2, 2)
	1 0 6
		('0 2 7', 46, 2, 2)
35
	0 4 0
		('1 2 0', 31, 2, 3)
	3 1 1
		('4 1 0', 36, 1, 3)
	0 0 0
		('0 0 0', 3, 1, 2)
	

D_curr : 17 10 0
Dprime : 17 10 0
total execution time : 57

  || ANALYSIS || 

TO EXPLORE : deque([])

Total time : 57
Success : False


#### Publication des résultats

In [58]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"" Catch me 2^k rounds"

[master 908100c] Automatic results update : Time_Bcast---rennes Catch me 2^k rounds
 Committer: Najwa Ez Zine <nezzine@frennes>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 22 files changed, 210 insertions(+), 263 deletions(-)
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-21/parasilo-26_2018-06-21_11:12.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-21/parasilo-26_2018-06-21_11:14.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-21/parasilo-26_2018-06-21_11:15.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-0

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)